In [2]:
from openai import OpenAI
import os
import pandas as pd
from IPython.display import Markdown, display
import json

# Config
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("API key not set.")
client = OpenAI(api_key=api_key)

# UI Dataframes: d1, d2, mom, heatmap_table, severity_counts_df
scores = pd.merge(d1,d2,how='inner',on=['cpeName','Title'])
scores.sort_values(by='riskScore').head(10)

cves = pd.read_csv('../data/vuln_catalogue_v2.csv')
df = pd.merge(cves,scores, how='inner', on=['Title','cpeName','cveID'])
top10 = df.copy()
top10.drop(columns=['Unnamed: 0', 'sid', 'WrittenAt','last_modified', 'vectorString', 
           'privilegesRequired', 'userInteraction', 'scope', 'integrityImpact', 
           'cwes', 'references', 'tags', 'full_json'], inplace=True,axis=1)
records = top10.to_dict(orient='records')


def explain_top_risks(df, client):
    risks_json = json.dumps(records, indent=2)

    # Construct the system and user prompts
    system_prompt = (
        "You are a cybersecurity expert specializing in vulnerability risk analysis "
        "and mitigation planning with a focus on NIST's Cybersecurity Framework 2.0."
    )

    user_prompt = (
        "Below is a list of the 10 highest-risk vulnerabilities (CVEs) affecting my assets. "
        "For each item, do the following:\n"
        "1. Briefly explain why this CVE poses a high risk based on its details.\n"
        "2. Suggest concise, actionable mitigation steps or best practices.\n"
        "Generate a report memo with only the subject line in the memo header,"
        "Present the output as a numbered list. Here are the vulnerabilities:\n"
        f"{risks_json}"
    )

    # Send to OpenAI
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=1000,
        temperature=0.2
    )
    return response.choices[0].message.content

# Now, call the function and store the result
top_risks = explain_top_risks(df, client)

with open("../markdown/top_risks.md", "w", encoding="utf-8") as f:
    f.write(top_risks)

with open("../markdown/top_risks.md", "r", encoding="utf-8") as f:
    md_content = f.read()

display(Markdown(md_content))

NameError: name 'd1' is not defined